In [1]:
#DANIELA ALEXANDRA DA SILVA 
#GUILHEREME SILVA MONTEIRO

In [2]:
%%sh
sudo pip install spark
sudo pip install pyspark


In [3]:
#Vamos importar as bibliotecas
#Pandas, podemos transformar um dataframe do pandas em um dataframe do spark e o contrário também
import pandas as pd

#Importando o spark e o pyspark
import spark,pyspark

#Importando as bibliotecas do pyspark.sql 
from pyspark.sql import *

#Importando as funções sql do spark
#documentação https://spark.apache.org/docs/latest/api/sql/index.html
from pyspark.sql import functions as f

#Importando os tipos de dados do spark
#documentação https://spark.apache.org/docs/latest/sql-ref-datatypes.html
from pyspark.sql import types as t

#Biblioteca datetime
from datetime import datetime, date
from pyspark.sql import types as t 
from pyspark.sql import functions as f

In [4]:
#2
spark = SparkSession.builder.master("local").appName("AC5 DataEng").getOrCreate()

In [5]:
#4
df_vendas = spark.read.format("parquet").option("header","true").option("delimiter",";").option("encoding", "UTF-8").load("/content/vendas.parquet")

In [ ]:
df_vendas.show()

In [7]:
#5
df_vendas2 = df_vendas.withColumn("data",df_vendas.data.cast(t.DateType()))


In [8]:
#6
#Criando novas colunas
df_vendas2 = df_vendas2.withColumn("DIA",f.dayofmonth("data"))
df_vendas2 = df_vendas2.withColumn("MES",f.month("data"))
df_vendas2 = df_vendas2.withColumn("ANO",f.year("data"))


In [ ]:
df_vendas2.show()

In [10]:
#7
df_filtro = df_vendas2.withColumn("SMS_TRATADO",f.when(df_vendas2.sms.isNull(),False).otherwise(df_vendas2.sms))



In [ ]:
df_filtro.show()

In [12]:
#8
df_filtro = df_vendas2.withColumn("total",df_vendas2.qtd * df_vendas2.vlr)

In [ ]:
df_filtro.show()

In [ ]:
#9
#Partition by irá criar uma arvore de pastas
df_filtro.write.partitionBy("ANO","MES","DIA").parquet("/content/vendas_tratadas")

In [ ]:
#10
#cria tabela temporaria 
df_filtro.registerTempTable("vendas_table")


In [ ]:
df_filtro.show()

In [17]:
df_1 = df_filtro

In [ ]:
#11
#query criando dataframe
df_1 = spark.sql("select distinct cidade from vendas_table" )
df_1.show()

In [ ]:
#Gravando dados como parquet
df_1.write.parquet("/content/cidade")

In [ ]:
#12
produto = df_filtro.select("des_produto","des_familia", "des_secao", "des_categoria","des_sub_categoria").distinct()
produto.distinct()
produto.show()

In [ ]:
#12
#Função Trim
produto = produto.withColumn("SEM ESPAÇO",f.trim(produto.des_familia)).distinct()
produto = produto.withColumn("SECAO",f.trim(produto.des_secao)).distinct()
produto = produto.withColumn("CATEGORIA",f.trim(produto.des_categoria)).distinct()
produto = produto.withColumn("SUB-CATEGORIA",f.trim(produto.des_sub_categoria)).distinct()

produto.show()

In [25]:
#13
venda_mulheres = df_filtro.where(df_filtro.sexo== 'F')

In [ ]:
venda_mulheres.show()

In [32]:
venda_mulheres.write.format("json").save("venda_mulher")


In [37]:
#14
vendas_unica = df_filtro.where((df_filtro.qtd == 1) & (df_filtro.vlr < 10))

In [39]:
vendas_unica.write.format("csv").save("venda_unica")
